In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/Chapter7/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/Chapter7


In [ ]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
sc =SparkContext()

In [ ]:
spark = SparkSession(sc)

In [ ]:
# Use Spark to read in the flights csv file.
data = spark.read.csv("flights.csv", inferSchema=True,header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [ ]:
print((data.count(), len(data.columns)))

(50000, 10)


In [ ]:
data.head(1)

[Row(mon=11, dom=20, dow=6, carrier='US', flight=19, org='JFK', mile=2153, depart=9.48, duration=351, delay='NA')]

In [ ]:
for item in data.head():
    print(item)

11
20
6
US
19
JFK
2153
9.48
351
NA


In [ ]:
data.columns

['mon',
 'dom',
 'dow',
 'carrier',
 'flight',
 'org',
 'mile',
 'depart',
 'duration',
 'delay']

In [ ]:
data.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



## Vector Assembler

In [ ]:
# Create an assembler
assembler = VectorAssembler(
                            inputCols=['mile'],
                            outputCol='features'
                            ) # inputs

In [ ]:
# Transfom data by assembler
data_pre = assembler.transform(data)

In [ ]:
# Data_pre after transfoming
data_pre.select('features').show(2,truncate=False)

+--------+
|features|
+--------+
|[2153.0]|
|[316.0] |
+--------+
only showing top 2 rows



In [ ]:
data_pre.show(2,False)

+---+---+---+-------+------+---+----+------+--------+-----+--------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|features|
+---+---+---+-------+------+---+----+------+--------+-----+--------+
|11 |20 |6  |US     |19    |JFK|2153|9.48  |351     |NA   |[2153.0]|
|0  |22 |2  |UA     |1107  |ORD|316 |16.33 |82      |30   |[316.0] |
+---+---+---+-------+------+---+----+------+--------+-----+--------+
only showing top 2 rows



=> Just one vectors instead of 4 columns

In [ ]:
# Just select features (after transforming from 4 columns) and output
final_data = data_pre.select('features','duration')

In [ ]:
final_data.show(2,truncate=False)

+--------+--------+
|features|duration|
+--------+--------+
|[2153.0]|351     |
|[316.0] |82      |
+--------+--------+
only showing top 2 rows



In [ ]:
final_data.count()

50000

In [ ]:
final_data = final_data.na.drop()
final_data.count()

50000

## Train, Test split

In [ ]:
# Train, Test split with ratio: 80:20
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [ ]:
train_data.describe().show()

+-------+-----------------+
|summary|         duration|
+-------+-----------------+
|  count|            39957|
|   mean|152.1894786895913|
| stddev|87.54674972920415|
|    min|               30|
|    max|              560|
+-------+-----------------+



In [ ]:
test_data.describe().show()

+-------+-----------------+
|summary|         duration|
+-------+-----------------+
|  count|            10043|
|   mean|150.0802549039132|
| stddev|85.00316506786241|
|    min|               30|
|    max|              560|
+-------+-----------------+



=> Train data and test data description are the same. Do not have an significant difference

=> Difference between mean and sttdev is high => Difficult to predict exactly

## Build model

In [ ]:
# Create a Linear Regression Model object
lr = LinearRegression(
                      labelCol='duration',
                      )
# => default: featuresCol='features', labelCol='label', predictionCol='prediction'

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
# Print the cofficients and intercept for linear regression
print('Coefficients : {} Intercept: {}'.format(lrModel.coefficients, lrModel.intercept))

Coefficients : [0.12156161289331949] Intercept: 44.45366551004834


## Model evaluation

In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
# Interesting results...
test_results.residuals.show(5)

+------------------+
|         residuals|
+------------------+
|-8.598293573900747|
|-6.598293573900747|
|-5.598293573900747|
|-5.598293573900747|
|-5.598293573900747|
+------------------+
only showing top 5 rows



In [ ]:
print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))
print('r2: {}'.format(test_results.r2))

RMSE: 17.025147101438353
MSE: 289.8556338256148
r2: 0.9598805659470504


=> RMSE = 17 << stddev = 85 => Model is good

In [ ]:
# check test dataset
test_model = lrModel.transform(test_data)

In [ ]:
# Inspect results
test_model.select('prediction', 'duration').show(5)

+-----------------+--------+
|       prediction|duration|
+-----------------+--------+
|52.59829357390075|      44|
|52.59829357390075|      46|
|52.59829357390075|      47|
|52.59829357390075|      47|
|52.59829357390075|      47|
+-----------------+--------+
only showing top 5 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
RegressionEvaluator(labelCol='duration').evaluate(test_model)

17.025147101438353

## Save and Load model

In [ ]:
# Save model
lrModel.save('lrModel_Flights_50k')

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

In [ ]:
# Load model
lrModel2 = LinearRegressionModel.load('lrModel_Flights_50k')

## New data prediction

In [ ]:
# Predict new values (assuming select test_data)
unlabeled_data = test_data.select('features')

In [ ]:
predictions = lrModel2.transform(unlabeled_data)

In [ ]:
predictions.show(5)

+--------+-----------------+
|features|       prediction|
+--------+-----------------+
|  [67.0]|52.59829357390075|
|  [67.0]|52.59829357390075|
|  [67.0]|52.59829357390075|
|  [67.0]|52.59829357390075|
|  [67.0]|52.59829357390075|
+--------+-----------------+
only showing top 5 rows



=> In fact, we have a new data and have to standardize and transform before predicting